In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/SEDS')


import myregex


Mounted at /content/drive


In [2]:
import re
import numpy as np
import pandas as pd


data_a = pd.read_excel("/content/drive/MyDrive/SEDS/PullRequest_Wajdi.xlsx")

# Dropping columns unused in our project
columns_to_drop = ['URL_1', 'Original_pull_request', 'committer name',
                   'Committer_wrote_description', 'Rest_of_comments', 'created_at', 'updated_at', 'closed_at',
                   'URL_2', 'Original_pull_request.1', 'committer name.1',
                   'Committer_wrote_description.1', 'Rest_of_comments.1', 'Updated_at_2', 'Updated_at_2.1', 'Closed_at_2',]
data_a.drop(columns_to_drop, axis=1, inplace=True)

# Renaming columns for paralelism and clarity
data_a.rename(columns={"Content": "Content_1", "Content.1": "Content_2"}, inplace=True)

# Add column 'Duplicates' to be used later
data_a["Duplicates"] = 0


In [3]:
# Data_a will provide the first pull request in each row, data_b will provide 
# the second. They'll be added back together once processed so that each PR
# ocupies a single row with an attribute 'Duplicates' indicating which other
# PR, if any, that PR is a duplicate of ('0' means none).
data_b = data_a.copy()

# Populate 'Duplicates' with the ID of the duplicated PR
data_a['Duplicates'] = np.where(data_a['Label'] == 1, data_a['Pull_id_2'], 0)
data_b['Duplicates'] = np.where(data_b['Label'] == 1, data_a['Pull_id_1'], 0)

# Drop now superfluous columns (which differes from a to b)


columns_to_drop = ['Pull_id_2', 'Title_2', 'Content_2', 'Files_2', 'Diff_info_2', 'Diff_content_2']
data_a.drop(columns_to_drop, axis=1, inplace=True)
columns_to_drop = ['Pull_id_1', 'Title_1', 'Content_1', 'Files_1', 'Diff_info_1', 'Diff_content_1']
data_b.drop(columns_to_drop, axis=1, inplace=True)


# Fix names of columns

data_a.rename(columns={"Pull_id_1": "Pull_id", "Title_1": "Title", "Content_1": "Content",
                       "Files_1": "Files", "Diff_info_1": "Diff_info", "Diff_content_1": "Diff_content"}, inplace=True)
data_b.rename(columns={"Pull_id_2": "Pull_id", "Title_2": "Title", "Content_2": "Content",
                       "Files_2": "Files", "Diff_info_2": "Diff_info", "Diff_content_2": "Diff_content"}, inplace=True)

# Concatinate the altered frames
result = pd.concat([data_a, data_b])

# Write the results for use elsewhere
result.to_csv('/content/drive/MyDrive/SEDS/PullRequeset_cleaned.csv',index_label=False,index=False)



In [ ]:
new_df = pd.read_csv("/content/drive/MyDrive/SEDS/PullRequeset_cleaned.csv")

In [ ]:
def get_added_comments(text):
  """This function will get both multiline and single line added comments"""

  first = myregex.find_only_multiline_comment(text)
  second = myregex.find_only_added_comments(text)
  return first + " " + second


In [ ]:
new_df['added_comment'] = new_df['Diff_content'].apply(get_added_comments)
new_df['removed_comment'] = new_df['Diff_content'].apply(myregex.find_only_removed_comments)
new_df['added_code'] = new_df['Diff_content'].apply(myregex.find_only_added_code)
new_df['removed_codo'] = new_df['Diff_content'].apply(myregex.find_only_removed_code)

In [ ]:
new_df.to_csv('/content/drive/MyDrive/SEDS/preprocessed.csv',index_label=False,index=False)

In [ ]:
new_df.head()

,Project,Pull_id,Title,Content,Files,Diff_info,Diff_content,Duplicates,label,added_comment,removed_comment,added_code,removed_codo
0,joomla/joomla-cms,6660.0,Undefined $model variable for postDeleteHook()...,$model is undefined at line 140 for $this->pos...,libraries/legacy/controller/admin.php,https://github.com/joomla/joomla-cms/pull/6660...,diff --git a/libraries/legacy/controller/admin...,7097.0,0,Invoke the postDelete method to allow for the...,Invoke the postDelete method to allow for the ...,$this->postDeleteHook($model$cid);,$this>postDeleteHook($model$cid);
1,joomla/joomla-cms,3088.0,[#33256] downloaded vcard is empty,http://joomlacode.org/gf/project/joomla/tracke...,components/com_contact/views/contact/view.vcf.php,https://github.com/joomla/joomla-cms/pull/3088...,diff --git a/components/com_contact/views/cont...,3114.0,0,Execute and display a template script @param ...,,*Executeanddisplayatemplatescript. *@paramstri...,$app>setHeader('Contentdisposition''attachment...
2,joomla/joomla-cms,3093.0,Bind Variable Fix Within PDO Driver,This fix is to revert back to original behavio...,libraries/joomla/database/driver/pdo.php,https://github.com/joomla/joomla-cms/pull/3093...,diff --git a/libraries/joomla/database/driver/...,3432.0,0,Create a stringified version of the query (wi...,Store reference to the JDatabaseQuery instance:,$sql=$this->replacePrefix((string)$query); $th...,$query=$this>replacePrefix((string)$query); $t...
3,joomla/joomla-cms,5664.0,CAPTCHA and reCAPTCHA (Legacy) #5595,We were wrong to call it nocaptcha. We were to...,administrator/language/en-GB/en-GB.com_config....,https://github.com/joomla/joomla-cms/pull/5664...,diff --git a/administrator/language/en-GB/en-G...,5888.0,0,"PLG_CAPTCHA_NOCAPTCHA_XML_DESCRIPTION=""This C...","PLG_CAPTCHA_NOCAPTCHA_XML_DESCRIPTION=""This CA...",,
4,joomla/joomla-cms,2596.0,Fix a failing unit test [#32813],This should fix a unit test that is currently ...,tests/unit/suites/libraries/joomla/registry/fo...,https://github.com/joomla/joomla-cms/pull/2596...,diff --git a/tests/unit/suites/libraries/jooml...,2612.0,0,Check for different PHP behavior of displayin...,,"$checkFalse='<check ?' :'>< $string=""<?xmlvers...","""<nodename=\""booleanfalse\""type=\""boolean\""><"
